### Local Alignment Algorithm
* BDPM(Bio Big Data and Precision Medicine) Training Course. (10/26/2018)
* Written by Jeongu Park (https://github.com/jeongu)

In [1]:
import numpy as np
import pandas as pd

class local_alignment:
    def __init__(self, match_award, mismatch_penalty, gap_penalty):
        self.match_award = match_award
        self.mismatch_penalty = mismatch_penalty
        self.gap_penalty = gap_penalty
        self.order_stack = []
        self.index_stack = []
        
    def check_match(self, data, i, j):
        if data.index[i] == data.columns[j]: return self.match_award
        else: return self.mismatch_penalty
    
    def make_matrix(self, first_seq, second_seq):
        first_seq2 = ["-"]
        second_seq2 = ["-"]
        first_seq2.extend(list(first_seq))
        second_seq2.extend(list(second_seq))
        
        data = [np.zeros(len(first_seq2), dtype='int32') for i in second_seq2]
        data = pd.DataFrame(data, columns=first_seq2, index=second_seq2)
        
        # save the route to trace back
        flag = []
        
        # fill out the matrix
        for i in range(1, data.shape[0]):
            temp = []
            for j in range(1, data.shape[1]):
                value_score = [data.iloc[i,j-1]+self.gap_penalty,
                               data.iloc[i-1,j-1]+self.check_match(data, i, j),
                               data.iloc[i-1,j]+self.gap_penalty,
                               0]
                data.iloc[i,j] = max(value_score)
                temp.append([data.iloc[i,j] == item for item in value_score])
            flag.append(temp)
        return data, flag
    
    def extract_max(self, data):
        temp = np.array(data.max())
        col = np.where(temp == temp.max())[0][0]
        temp = data.iloc[:,col]
        row = np.where(temp == temp.max())[0][0]
        return data.max().max(), row, col
        
    def extract_position(self, i, j, position):
        if position == 0: return [i,j-1]
        elif position == 1: return [i-1,j-1]
        elif position == 2: return [i-1,j]
        else: return None
    
    def stack_route(self, i, j, data, flag, point):
        term = flag[i][j]
        cnt = flag[i][j].count(True)
        
        # stack order
        for k in range(cnt): self.order_stack.append(point)
        
        # stack index
        if cnt == 1:
            pos = term.index(True)
            self.index_stack.append(self.extract_position(i, j, pos))
        elif cnt == 2:
            term = np.array(term)
            term = np.where(term == True)[0]
            self.index_stack.append(self.extract_position(i, j, term[0]))
            self.index_stack.append(self.extract_position(i, j, term[1]))
        elif cnt == 3:
            for k in range(3): self.index_stack.append(self.extract_position(i, j, k))
        else:
            print("error: 'cnt' is out of range (1,3)")
        return None
    
    def backtracking(self, data, flag, row, col):
        i = row - 1
        j = col - 1
        point = 0
        key = 0
        seq_dic = {str(key):[[i,j]]}
        
        # make index list
        while i >= 0 or j >= 0:
            self.stack_route(i, j, data, flag, point)
            self.order_stack.pop()
            term = self.index_stack.pop()
            i, j = term[0], term[1]
            if data.iloc[i+1,j+1] == 0: break
            seq_dic[str(key)].append(term)
            point += 1
            if len(self.order_stack) != 0 and i < 0 or j < 0:
                point = self.order_stack.pop()
                term = self.index_stack.pop()
                i, j = term[0], term[1]
                if data.iloc[i+1,j+1] == 0: break
                key += 1
                seq_dic[str(key)] = seq_dic[str(key-1)][:point+1]
                seq_dic[str(key)].append(term)
        
        # convert index list to name list
        key = 0
        for value in seq_dic.values():
            temp = [i for i in zip(*value)]
            row, col = temp[0], temp[1]
            row = [row[i] if row[i] != row[i+1] else -1 for i in range(len(row)-1)] + [row[-1]]
            col = [col[i] if col[i] != col[i+1] else -1 for i in range(len(col)-1)] + [col[-1]]
            
            row = [data.index[i+1] for i in row]
            col = [data.columns[i+1] for i in col]
            row = "".join(row)[::-1]
            col = "".join(col)[::-1]
            seq_dic[str(key)] = [col, row]
            key += 1
        return seq_dic
    
if __name__ == "__main__":
    # input data
    print("----------- local alignment -----------")
    first_seq = input("Please enter the first nucleotide sequence: ")
    second_seq = input("Please enter the second nucleotide sequence: ")
    '''
    first_seq = "AACTCACACTGTT"
    second_seq = "TGCACACG"
    '''
    # assign award and penalties
    match_award = 3
    mismatch_penalty = -1
    gap_penalty = -2
    
    # local alignment
    process = local_alignment(match_award, mismatch_penalty, gap_penalty)
    data, flag = process.make_matrix(first_seq, second_seq)
    score, row, col = process.extract_max(data)
    seq_dic = process.backtracking(data, flag, row, col)
    for key in seq_dic.keys():
        print("< case",int(key)+1,">")
        print("score:", score)
        print(seq_dic[key][0])
        print(seq_dic[key][1])

----------- local alignment -----------


Please enter the first nucleotide sequence:  AACTCACACTGTT
Please enter the second nucleotide sequence:  TGCACACG


< case 1 >
score: 17
T-CACACTG
TGCACAC-G


In [2]:
data

,-,A,A,C,T,C,A,C,A,C,T,G,T,T
-,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,3,1,0,0,0,0,3,1,3,3
G,0,0,0,0,1,2,0,0,0,0,1,6,4,2
C,0,0,0,3,1,4,2,3,1,3,1,4,5,3
A,0,3,3,1,2,2,7,5,6,4,2,2,3,4
C,0,1,2,6,4,5,5,10,8,9,7,5,3,2
A,0,3,4,4,5,3,8,8,13,11,9,7,5,3
C,0,1,2,7,5,8,6,11,11,16,14,12,10,8
G,0,0,0,5,6,6,7,9,10,14,15,17,15,13
